In [15]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import talib
from datetime import timedelta, datetime
import requests
import time
import pickle
from warnings import filterwarnings
from pykrx import stock
import subprocess
from bs4 import BeautifulSoup
import pandas_ta as ta
filterwarnings('ignore')

# bot.sendMessage(chat_id, 'test')
df_origin = pd.DataFrame()
df_origin = fdr.StockListing('KRX')
df_origin = df_origin[df_origin['Market'] != 'KONEX']
t = datetime.now()

tillDate = t + timedelta(days=-230)
tillDate = tillDate.strftime("%Y-%m-%d")
fin_df = pd.DataFrame()
for index, row in df_origin[['Name', 'Code']].iterrows():
    name, symbol = row['Name'], row['Code']
    # , '2022-05-01', '2022-11-10'
    try:
        df = fdr.DataReader(symbol, tillDate)

        df['symbol'] = symbol
        df['date'] = df['Open'].index

        df.ta.ema(append=True, length=112, col_name='112')
        # print(df)
        df['112_diff'] = ((df['Close'] - df['EMA_112']) / df['EMA_112']) * 100
        df['tradeMoney'] = df['Close'] * df['Volume']
        df.ta.sma(append=True, length=60, col_name='60')
        df.ta.sma(append=True, length=20, col_name='20')
        df.ta.sma(append=True, length=5, col_name='5')
        # print(df)
        df['MADiff'] = (df['SMA_20'] - df['SMA_5']) / df['SMA_5'] * 100
        df['6020Diff'] = (df['SMA_60'] - df['SMA_20']) / df['SMA_20'] * 100
        df['Close_diff'] = (df['Close'] - df['Open']) / df['Open'] * 100
        df['lg'] = (df['High'] - df['Close']) / df['Close'] * 100
        df['sh'] = (df['Open'] - df['Low']) / df['Low'] * 100
        df['idx'] = range(len(df['Close']))
        df['Name'] = name

        # 1차 필터링
        tmp = df[(df['Close_diff'] <= 5) & (df['Close_diff'] > 0) & (df['SMA_20'] >= df['Close']) & (df['SMA_5'] <= df['Open']) & (
            df['lg'] <= 2) & (df['sh'] <= 2) & (df['MADiff'] <= 5) & (df['Close'] > 1000)]

        fin_df = pd.concat([fin_df, tmp], axis=0)

        if len(fin_df) > 100:
            break
    except Exception as e:
        print(e)
        continue


start_nums = 0
max_list = ['0'] * start_nums
low_list = ['0'] * start_nums
tmp_df = pd.DataFrame()
for i in range(start_nums, len(fin_df['Close'])):
    tmp = fdr.DataReader(fin_df['symbol'].iloc[i])
    tmp['date'] = tmp['Open'].index
    tmp['idx'] = [i for i in range(len(tmp['Close']))]
    idx = tmp[tmp['date'] == fin_df['date'].iloc[i]]['idx'].iloc[0]

    prev_df = tmp.iloc[:idx]

    # 여기서 손익가/손절가 파악
    # 손절가/손익가 = min max 최대 15봉
    # fixed_income = tmp['High']
    max_income = max(prev_df['High'].iloc[-25:])
    low_stop = min(prev_df['Low'].iloc[-10:])
    max_list.append(max_income)
    low_list.append(low_stop)

fin_df['max'] = max_list
fin_df['min'] = low_list

fin_df['max'] = fin_df['max'].apply(lambda x: int(x))
fin_df['min'] = fin_df['min'].apply(lambda x: int(x))
fin_df['maxPct'] = round(
    (fin_df['max'] - fin_df['Close']) / fin_df['Close'] * 100, 2)
fin_df['minPct'] = round(
    (fin_df['min'] - fin_df['Close']) / fin_df['Close'] * 100, 2)

# # 지수 조건 추가 5 > 10 상승 조건일 때만
# trends = []
# for i in range(len(fin_df['Close'])):
#     trend = kospi[kospi['date'] == fin_df['date'].iloc[i]]

#     if len(trend) < 1:
#         trends.append('하락')
#         continue
#     trends.append(trend['trend'].iloc[0])
# fin_df['trend'] = trends

# 2차 필터링
fin_df = fin_df[(fin_df['MADiff'] < 5) & (fin_df['6020Diff'] > 0) & (fin_df['Close'] < 30000) & (
    fin_df['6020Diff'] > 5) & (fin_df['tradeMoney'] > 10000000) & (fin_df['maxPct'] >= 5)]
fin_df = fin_df.sort_values('date', ascending=False)


# 1차 기관 외국인 수급 60일치 잡아오고
merge_df = pd.DataFrame()
for i in range(len(fin_df['Close'])):
    date1 = date1 = (
        fin_df['date'].iloc[i] + timedelta(days=-60)).strftime('%Y%m%d')
    date2 = fin_df['date'].iloc[i].strftime(
        '%Y%m%d').replace('-', '')
    symbol = fin_df['symbol'].iloc[i]

    response = requests.get(f"https://open.shinhansec.com/goodicyber/mk/1206.jsp?date1={date1}&date2={date2}&TrKey=I&code={symbol}")
    soup = BeautifulSoup(response.text, 'html.parser')
    table_data = soup.find_all('td', {'align': 'center'})
    data_list = [td.get_text() for td in table_data]
    tables = pd.read_html(str(soup.find('table', class_='content1')), header=1)
    dfs = tables[0].dropna(inplace=True)
    time.sleep(0.3)

    # DataFrame 컬러명 수정
    dfs.columns = ['DATE', 'INDIVIDUAL', 'FOREIGN', 'INST', 'FINC_INST',
                        'IVST_INST', 'BANK', 'FINC_IVST', 'ISRNC', 'FINC_ETC', 'INST_ETC']
    dfs.drop('DATE', axis=1, inplace=True)
    merge_df = pd.concat(
        [merge_df, pd.DataFrame(dfs.sum()).T], axis=0)
    # df = df.join(merge_df, lsuffix='_caller', rsuffix='_other')

fin_df.reset_index(drop='index', inplace=True)
merge_df.reset_index(drop='index', inplace=True)
df = fin_df.join(
    merge_df, lsuffix='_caller', rsuffix='_other')

# 추가적 지표 구하기
# #1. 60일 거래량 누적합, 주가 & 거래 일자별로 나눈 값 = 해당 기간동안 거래된 주식 총 수
df['tradeAccured'] = 0
# df['ichimoku'] = 0
df['bollingerUpper'] = 0
df['bollingerLower'] = 0
df['macdTrend'] = 0
df['rsi'] = 0
df['close112Diff'] = 0
df['AROsc'] = 0
df['willr'] = 0
df['mfi'] = 0
df['beta'] = 0
df['corr'] = 0

kospi = stock.get_index_ohlcv_by_date(
    "19800104", "20221226", '1001')
kospi.columns = ['Open', 'High', 'Low', 'Close',
                    'Volume', 'tradeMoney', 'stockCapital']

df

AttributeError: 'NoneType' object has no attribute 'columns'

In [18]:
tables

[            날짜      개인    외국인계     기관계      증권      투신    은행    종금     보험  \
 0   2023/10/11 -170168   -9220  179352  129082    1435     0     0   1948   
 1   2023/10/10  -53598 -161441  215707  177311    4802     0  1268  -2260   
 2   2023/10/06  -66017  -25486   93268   84049    3669     0     0  -1092   
 3   2023/10/05   -5505  -53021   57922   42153    4332     0     1  -8836   
 4   2023/10/04  109260 -116893   -4253  -74265   11330   832   745   2633   
 5   2023/09/27  -63464  121751  -60989  -50694   21849    42    68   3234   
 6   2023/09/26  515614 -254853 -275823 -115747    4641 -9085     0    -43   
 7   2023/09/25  -31187   -5828   37248   91536   -5221     0     0  -7033   
 8   2023/09/22  128976    1437 -129271  -18077  -19942   832 -1130  -7072   
 9   2023/09/21  423290  -90178 -348178 -205420  -67685  4000     0  -3505   
 10  2023/09/20   -6880   -8345   20567    1100   -8781  3000  -663  -7655   
 11  2023/09/19  -39187   66795  -27211 -125259   30562  3000   

In [12]:
import requests
from bs4 import BeautifulSoup
response = requests.get("https://open.shinhansec.com/goodicyber/mk/1206.jsp?date1=20230812&date2=20231011&TrKey=I&code=323410")
soup = BeautifulSoup(response.text, 'html.parser')
table_data = soup.find_all('td', {'align': 'center'})
data_list = [td.get_text() for td in table_data]
data_list
soup
tables = pd.read_html(str(soup.find('table', class_='content1')), header=1)

# Assuming you want the first table
df = tables[0]

# Display the DataFrame
# print(df)
df

,날짜,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2023/10/11,-170168,-9220,179352,129082,1435,0,0,1948,4422,203
1,2023/10/10,-53598,-161441,215707,177311,4802,0,1268,-2260,10207,-553
2,2023/10/06,-66017,-25486,93268,84049,3669,0,0,-1092,-14051,-1658
3,2023/10/05,-5505,-53021,57922,42153,4332,0,1,-8836,500,224
4,2023/10/04,109260,-116893,-4253,-74265,11330,832,745,2633,43413,5695
5,2023/09/27,-63464,121751,-60989,-50694,21849,42,68,3234,-70986,2576
6,2023/09/26,515614,-254853,-275823,-115747,4641,-9085,0,-43,-98396,11853
7,2023/09/25,-31187,-5828,37248,91536,-5221,0,0,-7033,-68498,70
8,2023/09/22,128976,1437,-129271,-18077,-19942,832,-1130,-7072,-89046,-779
9,2023/09/21,423290,-90178,-348178,-205420,-67685,4000,0,-3505,-45446,452
